# **Insights Reporter Prototype Design using Google Gemini Free**

In [ ]:
# Chromadb, for creating knowledge base and generating embeddings
!pip install chromadb

# Google genai, for LLM access
!pip install google-api-core
!pip install -U -q "google-genai==1.7.0"

# IPython
!pip install ipython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 13.6 MB/s e

In [ ]:
!pip install --upgrade google-generativeai

In [ ]:
print("Importing libraries...")
import json  # Used in the judging process

# Chromadb, for creating knowledge base and generating embeddings
import chromadb

# Google genai, for LLM access
import google.generativeai as genai
from IPython.display import HTML, Markdown, display
from google.api_core import retry
from google.generativeai import types

# Retry on 429 or 503 errors
#is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# Patch the method with retry logic
#genai.GenerativeModel.generate_content = retry.Retry(
#    predicate=is_retriable)(genai.GenerativeModel.generate_content)

from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# For orchestration function
import sys  # Standard library for system-specific parameters and functions
import uuid # Standard library for generating unique identifiers
import time # Standard library for working with time and dates

import os
import time
import re
import random
from collections import defaultdict
from datetime import datetime
import csv # potential future use, though not active now

# Data Handling
import pandas as pd
# Progress Bar
from tqdm.notebook import tqdm

# Google Generative AI & API Error Handling
from google.api_core import exceptions as google_exceptions
from google.generativeai.types import HarmCategory, HarmBlockThreshold

Importing libraries...


In [ ]:
#Load the reference dataset and create it in required format

# If true, causes intermediate results to be printed to the console
debug_mode = True

In [ ]:
# Instantiate a ChromaDB client. The default Chroma client is ephemeral, meaning it will not save to disk.
chroma_client = chromadb.Client()

# Create a Collection
collection = chroma_client.create_collection(name="rag_collection")

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd # Import pandas for CSV handling
import numpy as np
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document # Import Document class
# Removed PyMuPDFLoader as we are handling CSV

from google.colab import files
uploaded_files = files.upload() #uploaded open source sales metrics dataset from Kaggle

docs = []
for filename, content in uploaded_files.items():
    # Assuming the CSV contains a column with text data, replace 'text_column_name' with the actual column name
    # Also, decode the bytes content to string
    df = pd.read_csv(pd.io.common.BytesIO(content), encoding='utf-8')
    df['Price_Int'] = df['Price'].str.replace('$', '').astype(float)
    df['Name'] = df['Name'].str.replace(' ', '').str.lower()
    # Convert relevant columns to string and combine them into a single text field per row, including column names
    df['combined_text'] = df.apply(lambda row: '|'.join([f"{col}: {row[col]}" for col in df.columns]), axis=1)
    docs.extend([str({'page_content': row['combined_text'], 'metadata': {'source': filename, 'row': i}}) for i, row in df.iterrows()])

print(docs[0])

Saving sales_archive_data_sample.csv to sales_archive_data_sample.csv
{'page_content': 'Sale_ID: 1001|Customer_ID: C001|Product_ID: P001|Date: 01/01/24|Quantity: 2|Price: $20.00|Discount: 0.05|Name: jackallen|Region: West Coast|Product: Widget A|Type: Electronics|Price_Int: 20.0', 'metadata': {'source': 'sales_archive_data_sample.csv', 'row': 0}}


In [ ]:
df.head(2)
df['sales'] = (df['Price_Int'] * df['Quantity'])*(1-df['Discount'])
df = df.drop_duplicates()
#df.head(2)
#df.agg({'Name':'nunique'}).reset_index()
check_temp = df.groupby(['Name']).agg({'sales':'sum', 'Product_ID':'nunique', 'Product':lambda x: ', '.join(x), 'Type':lambda x: ', '.join(x)}).reset_index()
check_temp.head(5)

,Name,sales,Product_ID,Product,Type
0,aliceadams,6685.115,27,"Accessory AD, Device AS, Widget A, Gadget L, A...","Accessories, Electronics, Electronics, Home Ap..."
1,aliceallen,6398.000,24,"Tool AQ, Tool AM, Appliance Q, Tool BA, Access...","Hardware, Hardware, Home Appliances, Hardware,..."
2,alicehall,8047.975,26,"Widget AE, Gadget AP, Tool W, Widget AO, Widge...","Electronics, Home Appliances, Hardware, Electr..."
3,alicehill,8042.735,31,"Device AI, Item P, Widget AE, Appliance Q, Dev...","Electronics, Office Supplies, Electronics, Hom..."
4,aliceking,20868.250,45,"Widget AO, Widget H, Tool M, Widget H, Device ...","Electronics, Electronics, Hardware, Electronic..."


In [ ]:
print(len(docs))

#ID list marker generation for Chromadb
id_list = []
for i in range(1, len(docs)+1):
    id_list.append(str("id"+str(i)))

print(id_list)

#Add to Chromadb
# Add documents to the Collection in batches
batch_size = 5000 # Set a batch size less than the max allowed (5461)
for i in range(0, len(docs), batch_size):
    batch_docs = docs[i:i + batch_size]
    batch_ids = id_list[i:i + batch_size]
    collection.add(documents=batch_docs, ids=batch_ids)
    print(f"Added batch {int(i/batch_size) + 1}/ {int(len(docs)/batch_size) + 1}")

print("Finished creating reference dataset for LLM")
print(collection.count())
print(collection)

6135
['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9', 'id10', 'id11', 'id12', 'id13', 'id14', 'id15', 'id16', 'id17', 'id18', 'id19', 'id20', 'id21', 'id22', 'id23', 'id24', 'id25', 'id26', 'id27', 'id28', 'id29', 'id30', 'id31', 'id32', 'id33', 'id34', 'id35', 'id36', 'id37', 'id38', 'id39', 'id40', 'id41', 'id42', 'id43', 'id44', 'id45', 'id46', 'id47', 'id48', 'id49', 'id50', 'id51', 'id52', 'id53', 'id54', 'id55', 'id56', 'id57', 'id58', 'id59', 'id60', 'id61', 'id62', 'id63', 'id64', 'id65', 'id66', 'id67', 'id68', 'id69', 'id70', 'id71', 'id72', 'id73', 'id74', 'id75', 'id76', 'id77', 'id78', 'id79', 'id80', 'id81', 'id82', 'id83', 'id84', 'id85', 'id86', 'id87', 'id88', 'id89', 'id90', 'id91', 'id92', 'id93', 'id94', 'id95', 'id96', 'id97', 'id98', 'id99', 'id100', 'id101', 'id102', 'id103', 'id104', 'id105', 'id106', 'id107', 'id108', 'id109', 'id110', 'id111', 'id112', 'id113', 'id114', 'id115', 'id116', 'id117', 'id118', 'id119', 'id120', 'id121', 'id122', 'id1

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 11.8MiB/s]


Added batch 1/ 2
Added batch 2/ 2
Finished creating reference dataset for LLM
6135
Collection(name=rag_collection)


In [ ]:
#Define prompts and create functions for the RAG pipeline

# System prompt, supplies basic context for the chatbot
def get_system_prompt():
    return "You are a helpful information answering and summarization bot embedded in a performance dashboard for a sales team. Your role is to provide accurate answers to user's questions about the total sales, cost, type of products, and customer names. As a Retrieval-Augmented Generation (RAG) chatbot, you will receive additional context with each user query to ensure relevant and detailed responses."

In [ ]:
# Functions applied before the knowledge retrieval process

# Original verbose prompt template
QUERY_REWRITE_PROMPT_1 = """Here is a user's prompt, could you please improve it if needed,
a) By extracting essential information b) By matching records based on context stated, For example,if they ask 'What are the total number of products bought by and total sales for johndoe? ', Do an exact search for the name johndoe in all documents and get documents only where the name is johndoe. Do not get any other documents. Assume johndoe has 3 products with Price_Int 20, Quantity 4, Discount 0, then calculate total_sales using (No. of products * ((Price_Int * Quantity) * (1-Discount))) as 240, and count the unique products as 3, Return the answer as 'johndoe bought $240 worth of products. He bought 3 products.'
    **Important: ** your answer will be given directly to an automated program, so **only** return the text of the improved prompt,
    nothing else.\n\nOK, now here is their prompt:\n\n{user_query}"""

# New concise prompt template
QUERY_REWRITE_PROMPT_2 = """Improve this user query for our knowledge base by:
1. Parsing the prompt to extract essential information
2. Expanding domain-specific context if needed and using the correct tool to calculate the answer
3. Maintaining the original intent and not changing the query too much

Query: {user_query}

**Important: ** your answer will be given directly to an automated program, so **only** return the text of the improved prompt,
nothing else. Make sure you use the correct tool for any calculation needed to generate the answer."""


def query_rewrite_1(chat_model_id: str, user_prompt_string: str) -> str:
    """
    Original prompt adjustment function with added error handling.
    Uses the more verbose original prompt template.

    Args:
        chat_model_id (str): ID of the chat model to use
        user_prompt_string (str): Original user prompt

    Returns:
        str: Improved prompt string
    """

    system_prompt = get_system_prompt()

    user_prompt_string = system_prompt + "\n\n" + QUERY_REWRITE_PROMPT_1.format(user_query=user_prompt_string)

    adjusted_prompt = genai.GenerativeModel(model_name=chat_model_id).generate_content(
        contents = user_prompt_string)

    return adjusted_prompt.text


def query_rewrite_2(chat_model_id: str, user_prompt_string: str) -> str:
    """
    New prompt adjustment function using the more concise prompt template.

    Args:
        chat_model_id (str): ID of the chat model to use
        user_prompt_string (str): Original user prompt

    Returns:
        str: Improved prompt string
    """

    system_prompt = get_system_prompt()

    user_prompt_string = system_prompt + "\n\n" + QUERY_REWRITE_PROMPT_2.format(user_query=user_prompt_string)

    adjusted_prompt = genai.GenerativeModel(model_name=chat_model_id).generate_content(
        contents = user_prompt_string)

    return adjusted_prompt.text

In [ ]:
# Define a function to retrieve one or more knowledge base items based on an embedding
# Returns a list variable with a row for each knowledge item returned, columns similarity and text

def getknowledge(prompt_string: object, num_results: object, model_id: object) -> object:
    results = collection.query(
        query_texts=[prompt_string],  # Chroma will embed this for you
        n_results=num_results  # how many results to return
    )

    assert isinstance(results, object)
    return results

In [ ]:
# Template for generation phase

GENERATION_PROMPT_TEMPLATE = """{system_prompt}

Please answer this question:

'{query_prompt}'

Using these blocks of context text, presented in order of decreasing relevance.
This RAG program is using a fixed value of k={num_results}, so be aware that
later context may not be truly relevant.

**Important:** Your answer will display in a text cell in a Markdown table, so format
the output as follows:
 1) Generate plain text output with no markdown fences or extra text.
 2) Space is limited—be succinct and keep answers to 75 words or less.

{prompt_context}"""

In [ ]:
# Query the llm for an answer to the user's question
import logging # Import logging module

def query_llm(chat_model_id, prompt_context, num_results, query_prompt):
    # The API key should be configured globally using genai.configure(api_key=GOOGLE_API_KEY)

    system_prompt = get_system_prompt()

    prompt_string = GENERATION_PROMPT_TEMPLATE.format(
        system_prompt=system_prompt,
        query_prompt=query_prompt,
        num_results=num_results,
        prompt_context=prompt_context
    )
    #print(prompt_string)

    model = genai.GenerativeModel(
            model_name=chat_model_id,
            system_instruction=system_prompt,
            tools=[{"code_execution": {}}])

    answer = model.generate_content(contents=prompt_string)
    generated_answer = answer.text

    return generated_answer

In [ ]:
# Template for the LLM‑as‑judge prompt
EVAL_PROMPT_TEMPLATE = """You are an impartial expert grader for retrieval‑augmented chatbots.
Your job is to evaluate the chatbot’s answer against the gold answer
and the judging instructions, then return a JSON object with four fields.

### Inputs
• Query: {query}

• Gold answer (reference): {gold_answer}

• Chatbot answer to grade: {chatbot_answer}

• Judging instructions (override anything else if conflicts arise):
  {judging_instructions}

### Rubric — score each dimension 1‑5
1 = very poor, 3 = acceptable, 5 = excellent.

1. Relevance – Does the chatbot answer address the user’s query?
2. Faithfulness – Is every factual statement supported by the gold answer
   (or clearly marked as outside scope)? Penalize hallucinations.
3. Completeness – Does the answer include all key elements required by the
   gold answer **and** satisfy the judging instructions (if provided)?

### Confidence
After scoring, give a **confidence** value 0.0‑1.0
(1.0 = absolutely certain your scores are correct; 0.0 = pure guess).

### Output — return **only** valid JSON in this exact schema, with NO markdown fences or extra text
{{
  "relevance": <integer 1‑5>,
  "faithfulness": <integer 1‑5>,
  "completeness": <integer 1‑5>,
  "confidence": <float between 0.0 and 1.0>
}}
No additional keys, text, or explanations.
Think silently before answering, but output only the JSON, with NO markdown fences or extra text."""

In [ ]:
# Query the llm for its judgment of the answer to the user's question
def judge_answer(chat_model_id, question, gold_answer,answer,instructions):
    prompt_string = EVAL_PROMPT_TEMPLATE.format(
        query=question,
        gold_answer=gold_answer,
        chatbot_answer=answer,
        judging_instructions=instructions
    )
    # print(prompt_string)

    judgment = genai.GenerativeModel(model_name=chat_model_id).generate_content(
        contents = prompt_string)

    judgment_answer = judgment.text

    return judgment_answer

In [ ]:
# Handle cases where the human must do the judging

# Suppose we are in a cell where we've determined
# there's no gold answer or the LLM’s self-reported
# confidence is below 0.2. We'll ask the human to judge.

def human_judgment_flow(
    gold_answer: str,
    chatbot_answer: str
):
    # Print context for the human
    if gold_answer:
        print("=== Gold (Reference) Answer ===")
        print(gold_answer)
    else:
        print("No gold answer was provided.")

    print("\n=== Chatbot Answer ===")
    print(chatbot_answer)

    print("\n=== Judging Instructions ===")
    print(
        "Please provide scores for:\n"
        "1) Relevance      (1-5): Does the answer address the user’s query?\n"
        "2) Faithfulness   (1-5): Is every factual statement supported by context?\n"
        "3) Completeness   (1-5): Does the answer cover all key elements requested?\n"
        "Finally, provide a confidence in your scoring (0.0 to 1.0)."
    )
    print("\n(Use whole numbers for the first three, e.g. 3, 4, 5, and a decimal for confidence.)\n")

    # Collect scores one-by-one
    relevance_str = input("Relevance (1-5): ").strip()
    faithfulness_str = input("Faithfulness (1-5): ").strip()
    completeness_str = input("Completeness (1-5): ").strip()
    conf_str = input("Confidence in your scoring (0.0-1.0): ").strip()

    # Convert to proper types
    try:
        relevance = int(relevance_str)
    except ValueError:
        relevance = None
    try:
        faithfulness = int(faithfulness_str)
    except ValueError:
        faithfulness = None
    try:
        completeness = int(completeness_str)
    except ValueError:
        completeness = None
    try:
        confidence = float(conf_str)
    except ValueError:
        confidence = None

    # Optional: Add quick checks on valid ranges
    if relevance not in [1,2,3,4,5]:
        print("Warning: Relevance out of range. Setting to None.")
        relevance = None
    if faithfulness not in [1,2,3,4,5]:
        print("Warning: Faithfulness out of range. Setting to None.")
        faithfulness = None
    if completeness not in [1,2,3,4,5]:
        print("Warning: Completeness out of range. Setting to None.")
        completeness = None
    if confidence is not None and not (0.0 <= confidence <= 1.0):
        print("Warning: Confidence out of range. Setting to None.")
        confidence = None

    # Return in the same structure we used
    result = {
        "relevance": relevance,
        "faithfulness": faithfulness,
        "completeness": completeness,
        "confidence": confidence
        }
    return json.dumps(result)

In [ ]:
# Basic orchestration process to carry out the RAG chatbot response to a user query and log the results

# This function executes the chatbot code. It performs these functions:
#   • Receives the user’s questions from the web services
#   • Processes them as needed
#   • Looks up relevant context in the knowledge base
#   • Queries ChatGPT to get an answer to the user’s question
#   • Hands that answer back to the web services for display to the user

def orchestration(config, test_results, debug=False):
    # Generate a unique Run ID and timestamp
    unique_run_id = str(uuid.uuid4())
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")  # Current timestamp

    # Configure Generative AI with the API key
    genai.configure(api_key=GOOGLE_API_KEY)

    # ----- Setup section -----

    # Print original prompt
    original_prompt_string = config ["question"]["query"]


    # ----- Pre-retrieval section -----

    #  Set the preretrieval function and chat model id
    pre_retrieval_function = config ["pre_retrieval"]["function"]
    pre_retrieval_chat_model_id = config ["pre_retrieval"]["model"]

    # Adjust the prompt string as needed to make it more compatible with the RAG database
    match pre_retrieval_function:
        case 'query_rewrite_1':
            final_prompt_string = query_rewrite_1(pre_retrieval_chat_model_id, original_prompt_string)
        case 'query_rewrite_2':
            final_prompt_string = query_rewrite_2(pre_retrieval_chat_model_id, original_prompt_string)
        case _:
            print(f"Pre_retrieval function not found: {pre_retrieval_function}")

    if debug:
        #print(final_prompt_string)
        display(Markdown(f"**Original Prompt:** {original_prompt_string}"))
        display(Markdown(f"**Rewritten Prompt:** {final_prompt_string}"))


    # ----- Retrieval section -----

    # Fetch context items that match the prompt string
    #  Set the embedding model id
    retrieval_function = config ["retrieval"]["function"]
    retrieval_embedding_model_id = config ["retrieval"]["model"]

    # Set k, the number of results, to return a constant k items of context from the knowledge base
    num_results = config ["retrieval"]["num_results"]
    if num_results == 0:
        num_results = 50

    # Returns a list variable "results" with a row for each knowledge item returned, with columns "similarity" and "text"
    match retrieval_function:
        case 'getknowledge':
            results = getknowledge(final_prompt_string, num_results, retrieval_embedding_model_id)
        case 'getknowledge_via_chunks':
            results = getknowledge_via_chunks(final_prompt_string, num_results, retrieval_embedding_model_id)
        case _:
            print(f"Retrieval function not found: {retrieval_function}")

    knowledge = ""
    for doc_group in results["documents"]:
        for doc in doc_group:
            knowledge += doc + "\n\n"

    if debug:
        #print(knowledge)
        display(Markdown("**Retrieved Knowledge**"))
        #display(Markdown(knowledge))


    # ----- Generation section -----

    #  Set the generation function and chat model id
    generation_function = config ["generation"]["function"]
    generation_chat_model_id = config ["generation"]["model"]

    # Get an answer from the LLM
    match generation_function:
        case 'query_llm':
            #answer = query_llm(generation_chat_model_id, knowledge, num_results, original_prompt_string)
            answer = query_llm(generation_chat_model_id, knowledge, num_results, final_prompt_string)
            display(Markdown(answer))
        case _:
            print(f"Generation function not found: {generation_function}")


    # ----- Judging section -----

    #  Set the judging function and chat model id
    judgment_function = config ["judging"]["function"]
    judgment_chat_model_id = config ["judging"]["model"]

    # Judge the answer
    gold_answer = config ["question"]["gold_answer"]
    # If a gold answer is provided
    if gold_answer != "":
        # Call an LLM to judge the chatbot answer by comparing it to the gold answer, taking into account the judging instructions
        instructions = config ["question"]["instructions"]

        match judgment_function:
            case 'judge_answer':
                judgment = judge_answer(judgment_chat_model_id, final_prompt_string, gold_answer,answer,instructions)
            case _:
                print(f"Judging function not found: {judgment_function}")

        # Clean up JSON result: take substring between first { and last }
        start = judgment.find("{")
        end   = judgment.rfind("}")
        if start == -1 or end == -1:
            raise json.JSONDecodeError("No JSON object found", judgment, 0)
        judgment = judgment[start : end+1]

        try:
            data = json.loads(judgment)
            relevance = data["relevance"]
            faithfulness = data["faithfulness"]
            completeness = data["completeness"]
            confidence = data["confidence"]
        except (json.JSONDecodeError, KeyError) as e:
            print("Error parsing judgment:", e)
            relevance = None
            faithfulness = None
            completeness = None
            confidence = None

        if debug:
            print("Relevance:", relevance)
            print("Faithfulness:", faithfulness)
            print("Completeness:", completeness)
            print("Confidence:", confidence)
            display(Markdown("**LLM Judgment:**"))
            display(Markdown(f"Relevance: {relevance}"))
            display(Markdown(f"Faithfulness: {faithfulness}"))
            display(Markdown(f"Completeness: {completeness}"))
            display(Markdown(f"Confidence: {confidence}"))

    # If a gold answer is NOT provided, or if the confidence is <0.2
    if gold_answer == "" or (confidence is not None and confidence < 0.2):
        # Ask the human to judge the answer
        human_judgment = human_judgment_flow(
            gold_answer=gold_answer,
            chatbot_answer=answer
        )
        data = json.loads(human_judgment)
        relevance = data["relevance"]
        faithfulness = data["faithfulness"]
        completeness = data["completeness"]
        confidence = data["confidence"]
        if debug:
            display(Markdown("**Human Judgment:**"))
            display(Markdown(f"Relevance: {relevance}"))
            display(Markdown(f"Faithfulness: {faithfulness}"))
            display(Markdown(f"Completeness: {completeness}"))
            display(Markdown(f"Confidence: {confidence}"))


    # Process the scores here...
    scores_composite = 0.4*relevance + 0.4*faithfulness + 0.2*completeness  # final score = 0.4*R + 0.4*F + 0.2*C

    # Log the results to the test_results list

    # Create the row as a list of values in the specified order
    results_row = [
        unique_run_id,
        timestamp,
        pre_retrieval_function,
        pre_retrieval_chat_model_id,
        retrieval_function,
        retrieval_embedding_model_id,
        generation_function,
        generation_chat_model_id,
        judgment_function,
        judgment_chat_model_id,
        original_prompt_string,
        final_prompt_string,
        answer,
        gold_answer,
        relevance,
        faithfulness,
        completeness,
        scores_composite,
        confidence
        # For future work:
        # pipeline_tokens_in,
        # pipeline_tokens_out,
        # pipeline_latency,
        # grading_tokens_in,
        # grading_tokens_out,
        # grading_latency
        # quality_per_dollar  # = final_score / (pipeline_tokens / 1000 \* 0.005)
    ]

    # Append the new row to the test_results list
    test_results.append(results_row)

    # ----- Return response section -----
    return answer

In [ ]:
#Main Code - Start defining inputs for the RAG pipeline and functions

# Query list, contains a list of rows each with a question, gold answer, and (optional) judging instructions -- just one sample here, can have more
query_list = [["What are the total number of products bought by and total sales for aliceking?","'aliceking bought $20868 worth of products. She bought 45 products.'","None."]]

# Set up model options for 4 test configurations (retrieval)
pre_retrieval_function_options = ["query_rewrite_1","query_rewrite_2"]
# Changed pre_retrieval_model_options to a single model
pre_retrieval_model_options = ["gemini-2.0-flash"]
retrieval_function_options = ["getknowledge"]
retrieval_model_options = ["chromadb_default"]
retrieval_num_results_options = [6135]

#Model options for generation
generation_function_options = ["query_llm"]
# generation_model_options = ["gemini-1.5-flash-8b","gemini-2.0-flash","gemini-2.0-flash-lite"]
generation_model_options = ["gemini-2.0-flash"]

#Model options for LLM-as-a-judge
judge_function_options = ["judge_answer"]
judge_model_options = ["gemini-2.0-flash", "gemini-2.0-flash-lite"]

In [ ]:
test_configurations = []

#  Loop through pre-retrieval functions
for pre_func in pre_retrieval_function_options:
    #  Loop through pre-retrieval models
    for pre_model in pre_retrieval_model_options:
        #  Loop through retrieval functions
        for ret_func in retrieval_function_options:
            #  Loop through retrieval models
            for ret_model in retrieval_model_options:
                #  Loop through number of results to retrieve
                for ret_results in retrieval_num_results_options:
                    #  Loop through generation functions
                    for gen_func in generation_function_options:
                        #  Loop through generation models
                        for gen_model in generation_model_options:
                            #  Loop through judging functions
                            for jud_func in judge_function_options:
                                #  Loop through judging models
                                for jud_model in judge_model_options:
                                    #  Add a line to the test_configs list
                                    # Each configuration is a tuple containing one option from each list
                                    test_configurations.append(
                                        (pre_func, pre_model, ret_func, ret_model, ret_results, gen_func, gen_model,jud_func,jud_model)
                                    )

# Optional: Print out the test configurations to verify
if debug_mode:
    for config in test_configurations:
       print(config)

('query_rewrite_1', 'gemini-2.0-flash', 'getknowledge', 'chromadb_default', 6135, 'query_llm', 'gemini-2.0-flash', 'judge_answer', 'gemini-2.0-flash')
('query_rewrite_1', 'gemini-2.0-flash', 'getknowledge', 'chromadb_default', 6135, 'query_llm', 'gemini-2.0-flash', 'judge_answer', 'gemini-2.0-flash-lite')
('query_rewrite_2', 'gemini-2.0-flash', 'getknowledge', 'chromadb_default', 6135, 'query_llm', 'gemini-2.0-flash', 'judge_answer', 'gemini-2.0-flash')
('query_rewrite_2', 'gemini-2.0-flash', 'getknowledge', 'chromadb_default', 6135, 'query_llm', 'gemini-2.0-flash', 'judge_answer', 'gemini-2.0-flash-lite')


In [ ]:
#Test the RAG pipeline implementation and see if it is invoking correct tools

# Set up a results list to log test results
RESULTS_HEADER = ["Unique Run ID", # Add a first row with column headings
    "Timestamp",
    "Pre-Retrieval Function",
    "Pre-Retrieval Chat Model ID",
    "Retrieval Function",
    "Retrieval Embedding Model ID",
    "Generation Function",
    "Generation Chat Model ID",
    "Judging Function",
    "Judging Chat Model ID",
    "Prompt String",
    "Final Prompt String",
    "Answer",
    "Sample Answer",
    "Relevance (0-5)",
    "Faithfulness (0-5)",
    "Completeness (0-5)",
    "Composite Score (0-5)",
    "Confidence In Score (0-1)"]

# Start the results table with *one* row – the header
test_results = [RESULTS_HEADER]

if debug_mode:
    print(test_results)

[['Unique Run ID', 'Timestamp', 'Pre-Retrieval Function', 'Pre-Retrieval Chat Model ID', 'Retrieval Function', 'Retrieval Embedding Model ID', 'Generation Function', 'Generation Chat Model ID', 'Judging Function', 'Judging Chat Model ID', 'Prompt String', 'Final Prompt String', 'Answer', 'Sample Answer', 'Relevance (0-5)', 'Faithfulness (0-5)', 'Completeness (0-5)', 'Composite Score (0-5)', 'Confidence In Score (0-1)']]


In [ ]:
# Loop through the list of test configurations, calling the orchestration function once for each configuration
debug_mode = True

for pre_function, pre_model, ret_function, ret_model, ret_results, gen_function, gen_model, jud_function, jud_model in test_configurations:

    # Loop through the list of questions and answers
    for query_val, gold_answer, instructions in query_list:

        # Set up the pipeline configuration
        this_config = {
            "question": {
                "query": query_val,
                "gold_answer": gold_answer,
                "instructions": instructions
            },

            "pre_retrieval": {
                "function": pre_function,
                "model": pre_model
            },

            "retrieval": {
                "function": ret_function,
                "model": ret_model,
                "num_results" : ret_results
            },

            "generation": {
                "function": gen_function,
                "model": gen_model
            },

            "judging": {
                "function": jud_function,
                "model": jud_model
            }
        }
        # "orchestration" will run the test for this configuration, and then add a line with results to test_results
        answer = orchestration(this_config, test_results, debug_mode)
        if hasattr(answer, 'tool_calls') and answer.tool_calls:
            print("Tool was invoked:", answer.tool_calls)


**Original Prompt:** What are the total number of products bought by and total sales for aliceking?

**Rewritten Prompt:** Find all records for the customer "aliceking". Calculate the total number of unique products bought by aliceking. For each product bought by aliceking, calculate the sales using the formula `Price_Int * Quantity * (1 - Discount)`. Sum the sales across all products to find the total sales for aliceking. Return the total number of products and the total sales for aliceking.


**Retrieved Knowledge**

```text
Based on the provided context, aliceking purchased two unique products, "Accessory AN" and "Accessory AX"

*   Accessory AN: 4 * 13.0 * (1 - 0.04) = 49.92
*   Accessory AX: 2 * 17.5 * (1 - 0.08) = 32.2

Total Sales: 49.92 + 32.2 = 82.12

Total Number of Products: 2
Total Sales for aliceking: $82.12
```

Relevance: 5
Faithfulness: 1
Completeness: 1
Confidence: 1.0


...

**LLM Judgment:**

Relevance: 5

Faithfulness: 1

Completeness: 3

Confidence: 1.0

In [ ]:
from IPython.display import Markdown, display

# Selected columns - display all columns by default
# selected_columns = [
#     'Pre-Retrieval Function',
#     'Generation Chat Model ID',
#     'Prompt String',
#     'Final Prompt String',
#     'Answer',
#     'Sample Answer',
#     'Relevance (0-5)',
#     'Faithfulness (0-5)',
#     'Completeness (0-5)',
#     'Composite Score (0-5)',
#     'Confidence In Score (0-1)'
# ]
selected_columns = test_results[0]


# Header and data rows
headers = test_results[0]
rows = test_results[1:]
selected_indexes = [headers.index(col) for col in selected_columns]

# Determine which columns to center
# Centered columns are all columns except the first one
centered_columns = set(selected_columns[1:])
separator_cells = [
    ":---:" if col in centered_columns else "---" for col in selected_columns
]

# Build the markdown string
markdown = "| " + " | ".join(selected_columns) + " |\n"
markdown += "| " + " | ".join(separator_cells) + " |\n"

for row in rows:
    selected_row = []
    for i in selected_indexes:
        cell_content = str(row[i]).replace("\n", " ")
        if selected_columns[selected_indexes.index(i)] == 'Answer':
            # Truncate to the last 1024 characters (approximating tokens)
            cell_content = cell_content[-512:]
        selected_row.append(cell_content)
    markdown += "| " + " | ".join(selected_row) + " |\n"

# Display it
safe_md = markdown.replace("$", r"\$") # This keeps '$' characters from messing up the table display
display(Markdown(safe_md))

| Unique Run ID | Timestamp | Pre-Retrieval Function | Pre-Retrieval Chat Model ID | Retrieval Function | Retrieval Embedding Model ID | Generation Function | Generation Chat Model ID | Judging Function | Judging Chat Model ID | Prompt String | Final Prompt String | Answer | Sample Answer | Relevance (0-5) | Faithfulness (0-5) | Completeness (0-5) | Composite Score (0-5) | Confidence In Score (0-1) |
| --- | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 54705aa5-0bc8-4eab-a080-0e26cc0f7604 | 2025-08-09 06:55:06 | query_rewrite_1 | gemini-2.0-flash | getknowledge | chromadb_default | query_llm | gemini-2.0-flash | judge_answer | gemini-2.0-flash | What are the total number of products bought by and total sales for aliceking? | Find all records for the customer "aliceking". Calculate the total number of unique products bought by aliceking. For each product bought by aliceking, calculate the sales using the formula `Price_Int * Quantity * (1 - Discount)`. Sum the sales across all products to find the total sales for aliceking. Return the total number of products and the total sales for aliceking.  | ```text Based on the provided context, aliceking purchased two unique products, "Accessory AN" and "Accessory AX"  *   Accessory AN: 4 * 13.0 * (1 - 0.04) = 49.92 *   Accessory AX: 2 * 17.5 * (1 - 0.08) = 32.2  Total Sales: 49.92 + 32.2 = 82.12  Total Number of Products: 2 Total Sales for aliceking: \$82.12 ``` | 'aliceking bought \$20868 worth of products. She bought 45 products.' | 5 | 1 | 1 | 2.6 | 1.0 |
| 952d5ee1-1b37-48aa-90a0-f7a4507d014d | 2025-08-09 06:55:21 | query_rewrite_1 | gemini-2.0-flash | getknowledge | chromadb_default | query_llm | gemini-2.0-flash | judge_answer | gemini-2.0-flash-lite | What are the total number of products bought by and total sales for aliceking? | Find all records for the customer "aliceking". Calculate the total number of unique products bought by aliceking. Calculate the total sales for aliceking using the formula (Number of products * ((Price_Int * Quantity) * (1-Discount))) for each product and summing the results. Return a string stating "aliceking bought [total sales amount] worth of products. She bought [total number of unique products] products."  | I cannot fulfill this request. The context does not contain sufficient information about the customer "aliceking". It does not include the products she bought, the quantities, or the prices. Therefore, I cannot calculate the total number of unique products bought by her or the total sales amount. | 'aliceking bought \$20868 worth of products. She bought 45 products.' | 1 | 5 | 1 | 2.6 | 1.0 |
| 6991c000-f69b-4b41-b7e7-85ef5405538d | 2025-08-09 06:55:47 | query_rewrite_2 | gemini-2.0-flash | getknowledge | chromadb_default | query_llm | gemini-2.0-flash | judge_answer | gemini-2.0-flash | What are the total number of products bought by and total sales for aliceking? | Fetch the total number of products purchased and calculate the total sales amount for customer "aliceking" using the 'sales_data' tool.  | lumns.str.replace(":.*", "", regex=True)  df['Quantity'] = df['Quantity'].str.extract('(\d+)').astype(int) df['Price_Int'] = df['Price_Int'].str.extract('(\d+\.\d+)').astype(float)   total_products = df['Quantity'].sum() total_sales = (df['Quantity'] * df['Price_Int']).sum()  print(f"Total Products Purchased: {total_products}") print(f"Total Sales Amount: \${total_sales:.2f}")   ``` ``` Total Products Purchased: 163 Total Sales Amount: \$4921.50  ``` Total Products Purchased: 163 Total Sales Amount: \$4921.50  | 'aliceking bought \$20868 worth of products. She bought 45 products.' | 5 | 1 | 1 | 2.6 | 1.0 |
| 22852f31-9921-494f-a119-8c85235dfd85 | 2025-08-09 06:58:33 | query_rewrite_2 | gemini-2.0-flash | getknowledge | chromadb_default | query_llm | gemini-2.0-flash | judge_answer | gemini-2.0-flash-lite | What are the total number of products bought by and total sales for aliceking? | Fetch the customer ID for "aliceking". Then, using the customer ID, retrieve the total number of products purchased and the total sales amount from the sales database. Display the total number of products and the total sales amount for that customer.  | ```text Customer ID for aliceking: C130. Total products purchased by aliceking: 36. Total sales amount for aliceking: \$608.00. ``` | 'aliceking bought \$20868 worth of products. She bought 45 products.' | 5 | 1 | 3 | 3.0 | 1.0 |
